## Tensor Import

In [2]:
import torch

## 1. Tensor Basics

In [3]:
x = torch.tensor(5)
print(x)

x = torch.tensor((1,2))
print(x)

x = torch.empty(2,3)
print(x)

tensor(5)
tensor([1, 2])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [4]:
# check size
print("x.size(): ", x.size())
print("x.shape: ", x.shape)

x.size():  torch.Size([2, 3])
x.shape:  torch.Size([2, 3])


In [5]:
# check data type
print(x.dtype)

torch.float32


In [6]:
# requires_grad argument
# This will tell pytorch that it will need to calculate the gradients for this tensor
# later in your optimization steps
# e.g. this is a variable in your model that you want to optimize
x = torch.tensor([5.5, 3], requires_grad=True)
print(x)

tensor([5.5000, 3.0000], requires_grad=True)


### Operations with Tensors

In [7]:
# Operations
x = torch.ones(2, 3)
y = torch.rand(2, 3)

# addition
z = x + y

# subtraction
z = x - y

# multiplication
z = x * y

# division
z = x / y

print(x)
print(y)
print(z)



tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.0169, 0.8968, 0.4759],
        [0.2063, 0.8035, 0.1852]])
tensor([[59.2106,  1.1151,  2.1015],
        [ 4.8474,  1.2446,  5.3987]])


In [8]:
# Slicing
x = torch.rand(3, 3)
print(x)

print("x[:, 0]", x[:, 0])
print("x[0, :]", x[0, :])
print("x[0, 0]", x[0, 0])

# Get the actual value if only 1 element in your tensor
print("x[0, 0].item()", x[0, 0].item())

tensor([[0.4051, 0.0538, 0.8213],
        [0.2421, 0.0257, 0.8906],
        [0.6525, 0.9393, 0.1267]])
x[:, 0] tensor([0.4051, 0.2421, 0.6525])
x[0, :] tensor([0.4051, 0.0538, 0.8213])
x[0, 0] tensor(0.4051)
x[0, 0].item() 0.4050580859184265


In [9]:
# Reshape with torch.view()
x = torch.rand(4, 4)
y = x.view(16)
z = x.view(-1, 8) # if -1 pytorch will automatically determine the necessary size

print(x)
print(y)
print(z)

tensor([[0.3140, 0.4942, 0.1982, 0.9171],
        [0.7565, 0.5525, 0.1197, 0.1299],
        [0.7275, 0.1836, 0.5662, 0.4160],
        [0.9562, 0.3117, 0.2850, 0.1574]])
tensor([0.3140, 0.4942, 0.1982, 0.9171, 0.7565, 0.5525, 0.1197, 0.1299, 0.7275,
        0.1836, 0.5662, 0.4160, 0.9562, 0.3117, 0.2850, 0.1574])
tensor([[0.3140, 0.4942, 0.1982, 0.9171, 0.7565, 0.5525, 0.1197, 0.1299],
        [0.7275, 0.1836, 0.5662, 0.4160, 0.9562, 0.3117, 0.2850, 0.1574]])


### NumPy

Converting a Torch Tensor to a NumPy array and vice versa

In [10]:
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)
print(type(b))

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
<class 'numpy.ndarray'>


In [11]:
# Careful: If the Tensor is on the CPU (not the GPU),
# both objects will share the same memory location, so changing one
# will also change the other
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [12]:
# numpy to torch with .from_numpy(x), or torch.tensor() to copy it
import numpy as np

a = np.ones(5)
b = torch.from_numpy(a) # they share the same memory location
c = torch.tensor(a)
print(a)
print(b)
print(c)

# again be careful when modifying
a += 1
print(a)
print(b)
print(c)

[1. 1. 1. 1. 1.]
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


### GPU Support

By default all tensors are created on the CPU. But we can also move them to the GPU (if it's available ), or create them directly on the GPU.

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.rand(2, 2).to(device) # move tensors to GPU device if available

# it is a small optimizations
x = torch.rand(2, 2, device=device)  # or directy create them on GPU if available

## 2. Autograd

The autograd package provides automatic differentiation for all operations on Tensors. Generally speaking, torch.autograd is an engine for computing the vector-Jacobian product. It computes partial derivates while applying the chain rule.

In [14]:
# requires_grad = True -> tracks all operations on the tensor
x = torch.rand(3, requires_grad=True)
y = x + 2

# y was created as a result of an operation, so it has a grad_fn attribute.
# grad_fn: references a Function that has created the Tensor
print(x) # created by the user -> grad_fn is None
print(y)
print(y.grad_fn)

tensor([0.0478, 0.3619, 0.7200], requires_grad=True)
tensor([2.0478, 2.3619, 2.7200], grad_fn=<AddBackward0>)


In [15]:
# more operations on y
z = y * y * 3
print(z)

z = z.mean()
print(z)

tensor([12.5802, 16.7353, 22.1957], grad_fn=<MulBackward0>)
tensor(17.1704, grad_fn=<MeanBackward0>)


In [16]:
# Compute the gradients with backpropagation
# When we finish our computation we can call .backward() and have all the gradients computed automatically.
# The gradient for this tensor will be accumulated into .grad attribute.
# It is the partial derivate of the function w.r.t. the tensor

z.backward()
print(x.grad) # dz/dx

# !!! Careful!!! backward() accumulates the gradient for this tensor into .grad attribute.
# !!! We need to be careful during optimization !!! optimizer.zero_grad()

tensor([4.0956, 4.7237, 5.4401])


### Stop a tensor from tracking history:

For example during the training loop when we want to update our weights, or after training during evaluation. These operations should not be part of the gradient computation. To prevent this, we can use:

* <code>x.requires_grad_(False)</code>
* <code>x.detach()</code>
* wrap in <code>with torch.no_grad():</code>

In [21]:
# .requires_grad_(...) changes an existing flag in-place.
a = torch.rand(2, 2)
b = (a * a).sum()
print(a.requires_grad)
print(b.grad_fn)

a.requires_grad_(True)
b = (a * a).sum()
print(a.requires_grad)
print(b.grad_fn)

False
None
True


In [23]:
# .detach(): get a new Tensor with the same content but no gradient computation:
a = torch.rand(2, 2, requires_grad=True)
b = a.detach()

print(a.requires_grad)
print(b.requires_grad)

True
False


In [26]:
# wrap in 'with torch.no_grad():'
a = torch.rand(2, 2, requires_grad=True)
print(a.requires_grad)

with torch.no_grad():
    b = a ** 2
    print(b.requires_grad)

True
False
